# Αυτό είναι ενα POC prework των δεδομένων πριν την αυτοματοποίηση.
## Παίρνουμε δεδομένα από το Hacker News API (Firebase).

### Το HN API είναι read-only JSON endpoints.

### Base URL

    https://hacker-news.firebaseio.com/v0/

### Top stories IDs:
    https://hacker-news.firebaseio.com/v0/topstories.json

### Details για ένα story (με id):
    https://hacker-news.firebaseio.com/v0/item/<ID>.json

### Δεν θελει API key. Απλά HTTP GET.

### Σημείωση:
### Το ranking των topstories ορίζεται από τον αλγόριθμο του Hacker News, ο οποίος λαμβάνει υπόψη κυρίως:

    - score (upvotes)

    - age/time (πόσο πρόσφατο είναι)

    - engagement / activity

# 1)Extract: 
##### Παίρνω details για τα πρώτα 20 stories από το topstories endpoint. (κρατάμε μόνο type = story)

In [18]:
!pip -q install requests pandas

In [25]:
import time
import requests
from typing import Optional, Dict, List, Any

BASE = "https://hacker-news.firebaseio.com/v0"
session = requests.Session()

def fetch_item(item_id: int) -> Optional[Dict[str, Any]]:
    r = session.get(f"{BASE}/item/{item_id}.json", timeout=30)
    r.raise_for_status()
    return r.json()

def fetch_top_ids() -> List[int]:
    r = session.get(f"{BASE}/topstories.json", timeout=30)
    r.raise_for_status()
    return r.json()

def top_20_stories_only(pause: float = 0.05) -> List[Dict[str, Any]]:
    top_ids = fetch_top_ids()  # up to 500

    stories: List[Dict[str, Any]] = []

    for item_id in top_ids:
        data = fetch_item(item_id)
        time.sleep(pause)

        if not data:
            continue


        if data.get("type") != "story":
            continue

        # πεταω dead/deleted
        if data.get("deleted") or data.get("dead"):
            continue

        stories.append(data)

        # stop όταν συμπληρωσουμε 20
        if len(stories) == 20:
            break

    return stories

top20 = top_20_stories_only()
[(s["id"], s.get("score"), s.get("title")) for s in top20[:5]]

[(46564696, 70, 'Allow me to introduce, the Citroen C15'),
 (46565132,
  34,
  'A Eulogy for Dark Sky, a Data Visualization Masterpiece (2023)'),
 (46560445,
  521,
  '“Erdos problem #728 was solved more or less autonomously by AI”'),
 (46524379, 190, 'Changes to Android Open Source Project'),
 (46565281, 37, 'All my new code will be closed-source from now on')]

In [26]:
### Σε Dataframe:
import pandas as pd

df_top20 = pd.DataFrame(top20)

cols = ["id", "score", "title", "url", "type", "by", "time", "descendants"]
df_top20 = df_top20.reindex(columns=cols)

df_top20

,id,score,title,url,type,by,time,descendants
0,46564696,70,"Allow me to introduce, the Citroen C15",https://eupolicy.social/@jmaris/11586059523809...,story,colinprince,1768043535,35
1,46565132,34,"A Eulogy for Dark Sky, a Data Visualization Ma...",https://nightingaledvs.com/dark-sky-weather-da...,story,skadamat,1768047800,18
2,46560445,521,“Erdos problem #728 was solved more or less au...,https://mathstodon.xyz/@tao/115855840223258103,story,cod1r,1767998355,284
3,46524379,190,Changes to Android Open Source Project,https://source.android.com/,story,TechTechTech,1767778546,112
4,46565281,37,All my new code will be closed-source from now on,https://twitter.com/MarcJSchmidt/status/200968...,story,mvelbaum,1768049485,33
5,46511555,158,Greenland sharks maintain vision for centuries...,https://phys.org/news/2026-01-eye-greenland-sh...,story,pseudolus,1767703492,62
6,46564762,13,New information extracted from Snowden PDFs th...,https://libroot.org/posts/going-through-snowde...,story,libroot,1768044239,4
7,46557489,281,JavaScript Demos in 140 Characters,https://beta.dwitter.net,story,themanmaran,1767984510,53
8,46556695,321,How Markdown took over the world,https://www.anildash.com/2026/01/09/how-markdo...,story,zdw,1767981140,246
9,46560217,172,Start your meetings at 5 minutes past,https://philipotoole.com/start-your-meetings-a...,story,otoolep,1767997143,157


# 2)Transform:

In [48]:
# Unix time σε datetime
df_top20["published_at"] = pd.to_datetime(df_top20["time"], unit="s", utc=True)
df_top20[["id", "time", "published_at"]].head(5)

,id,time,published_at
0,46564696,1768043535,2026-01-10 11:12:15+00:00
1,46565132,1768047800,2026-01-10 12:23:20+00:00
2,46560445,1767998355,2026-01-09 22:39:15+00:00
3,46524379,1767778546,2026-01-07 09:35:46+00:00
4,46565281,1768049485,2026-01-10 12:51:25+00:00


In [49]:
# Domain από URL
from urllib.parse import urlparse
import numpy as np

def extract_domain(u):
    # χειρισμός None/NaN/non-string/empty
    if not isinstance(u, str) or u.strip() == "":
        return None
    
    parsed = urlparse(u.strip())
    host = parsed.netloc.lower()

    # αν το url είναι "news.ycombinator.com/item?id=..." χωρίς scheme,
    # urlparse μπορεί να το διαβάσει περίεργα. Καλύπτουμε κι αυτό:
    if host == "" and "://" not in u:
        parsed2 = urlparse("https://" + u.strip())
        host = parsed2.netloc.lower()

    # αφαιρούμε "www."
    if host.startswith("www."):
        host = host[4:]

    return host or None

df_top20["domain"] = df_top20["url"].apply(extract_domain)

df_top20[["id", "title", "url", "domain"]].head(10)

,id,title,url,domain
0,46564696,"Allow me to introduce, the Citroen C15",https://eupolicy.social/@jmaris/11586059523809...,eupolicy.social
1,46565132,"A Eulogy for Dark Sky, a Data Visualization Ma...",https://nightingaledvs.com/dark-sky-weather-da...,nightingaledvs.com
2,46560445,“Erdos problem #728 was solved more or less au...,https://mathstodon.xyz/@tao/115855840223258103,mathstodon.xyz
3,46524379,Changes to Android Open Source Project,https://source.android.com/,source.android.com
4,46565281,All my new code will be closed-source from now on,https://twitter.com/MarcJSchmidt/status/200968...,twitter.com
5,46511555,Greenland sharks maintain vision for centuries...,https://phys.org/news/2026-01-eye-greenland-sh...,phys.org
6,46564762,New information extracted from Snowden PDFs th...,https://libroot.org/posts/going-through-snowde...,libroot.org
7,46557489,JavaScript Demos in 140 Characters,https://beta.dwitter.net,beta.dwitter.net
8,46556695,How Markdown took over the world,https://www.anildash.com/2026/01/09/how-markdo...,anildash.com
9,46560217,Start your meetings at 5 minutes past,https://philipotoole.com/start-your-meetings-a...,philipotoole.com


In [51]:
# Missing values check
missing = df_top20[["score","title","url","domain","by","time","descendants"]].isna().sum()
missing

score          0
title          0
url            1
domain         1
by             0
time           0
descendants    0
dtype: int64

In [52]:
# Duplicates check
dupe_ids = df_top20["id"].duplicated().sum()
dupe_ids

0

In [53]:
# Τύποι δεδομένων για την Postgres
df_top20.dtypes

id                            int64
score                         int64
title                        object
url                          object
type                         object
by                           object
time                          int64
descendants                   int64
published_at    datetime64[ns, UTC]
domain                       object
dtype: object

In [54]:
# sanity checks
checks = {
    "rows": len(df_top20),
    "unique_ids": df_top20["id"].nunique(),
    "min_score": df_top20["score"].min(skipna=True),
    "max_score": df_top20["score"].max(skipna=True),
    "missing_url": int(df_top20["url"].isna().sum()),
    "missing_descendants": int(df_top20["descendants"].isna().sum()),
    "types": df_top20["type"].value_counts(dropna=False).to_dict(),
}
checks

{'rows': 20,
 'unique_ids': 20,
 'min_score': 13,
 'max_score': 576,
 'missing_url': 1,
 'missing_descendants': 0,
 'types': {'story': 20}}

In [47]:
# Τελική μορφή στηλών
final_cols = ["id", "score", "title", "domain", "url", "type", "by", "published_at", "descendants"]
df_final = df_top20.reindex(columns=final_cols).copy()

df_final.head()

,id,score,title,domain,url,type,by,published_at,descendants
0,46564696,70,"Allow me to introduce, the Citroen C15",eupolicy.social,https://eupolicy.social/@jmaris/11586059523809...,story,colinprince,2026-01-10 11:12:15+00:00,35
1,46565132,34,"A Eulogy for Dark Sky, a Data Visualization Ma...",nightingaledvs.com,https://nightingaledvs.com/dark-sky-weather-da...,story,skadamat,2026-01-10 12:23:20+00:00,18
2,46560445,521,“Erdos problem #728 was solved more or less au...,mathstodon.xyz,https://mathstodon.xyz/@tao/115855840223258103,story,cod1r,2026-01-09 22:39:15+00:00,284
3,46524379,190,Changes to Android Open Source Project,source.android.com,https://source.android.com/,story,TechTechTech,2026-01-07 09:35:46+00:00,112
4,46565281,37,All my new code will be closed-source from now on,twitter.com,https://twitter.com/MarcJSchmidt/status/200968...,story,mvelbaum,2026-01-10 12:51:25+00:00,33


In [55]:
# Naming για Postgres
df_pg = df_final.rename(columns={
    "id": "story_id",
    "by": "author",
    "descendants": "comment_count",
    "published_at": "published_at_utc",
})

df_pg.head()

,story_id,score,title,domain,url,type,author,published_at_utc,comment_count
0,46564696,70,"Allow me to introduce, the Citroen C15",eupolicy.social,https://eupolicy.social/@jmaris/11586059523809...,story,colinprince,2026-01-10 11:12:15+00:00,35
1,46565132,34,"A Eulogy for Dark Sky, a Data Visualization Ma...",nightingaledvs.com,https://nightingaledvs.com/dark-sky-weather-da...,story,skadamat,2026-01-10 12:23:20+00:00,18
2,46560445,521,“Erdos problem #728 was solved more or less au...,mathstodon.xyz,https://mathstodon.xyz/@tao/115855840223258103,story,cod1r,2026-01-09 22:39:15+00:00,284
3,46524379,190,Changes to Android Open Source Project,source.android.com,https://source.android.com/,story,TechTechTech,2026-01-07 09:35:46+00:00,112
4,46565281,37,All my new code will be closed-source from now on,twitter.com,https://twitter.com/MarcJSchmidt/status/200968...,story,mvelbaum,2026-01-10 12:51:25+00:00,33


# 3)Load:

In [56]:
!pip -q install psycopg2-binary

In [64]:
# Connection
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port=15432,
    dbname="hn_db",
    user="hn_user",
    password="hn_pass"
)

with conn.cursor() as cur:
    cur.execute("SELECT now(), current_database(), current_user;")
    print(cur.fetchone())

(datetime.datetime(2026, 1, 10, 18, 47, 48, 108406, tzinfo=datetime.timezone.utc), 'hn_db', 'hn_user')


In [65]:
from psycopg2.extras import execute_batch

# 1) Create table (just in case)
create_sql = """
CREATE TABLE IF NOT EXISTS news_archive (
  story_id BIGINT PRIMARY KEY,
  title TEXT,
  url TEXT,
  domain TEXT,
  author TEXT,
  published_at_utc TIMESTAMPTZ,
  score INTEGER,
  comment_count INTEGER,
  first_seen_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
  last_seen_at  TIMESTAMPTZ NOT NULL DEFAULT NOW()
);
"""

with conn.cursor() as cur:
    cur.execute(create_sql)
conn.commit()

# 2) UPSERT:
# - first_seen_at: set only on INSERT (NOW())
# - last_seen_at: updated every run (NOW())
upsert_sql = """
INSERT INTO news_archive (
  story_id, title, url, domain, author, published_at_utc, score, comment_count,
  first_seen_at, last_seen_at
)
VALUES (
  %(story_id)s, %(title)s, %(url)s, %(domain)s, %(author)s, %(published_at_utc)s,
  %(score)s, %(comment_count)s,
  NOW(), NOW()
)
ON CONFLICT (story_id)
DO UPDATE SET
  title = EXCLUDED.title,
  url = EXCLUDED.url,
  domain = EXCLUDED.domain,
  author = EXCLUDED.author,
  published_at_utc = EXCLUDED.published_at_utc,
  score = EXCLUDED.score,
  comment_count = EXCLUDED.comment_count,
  last_seen_at = NOW();
"""

records = df_pg.to_dict(orient="records")

with conn.cursor() as cur:
    execute_batch(cur, upsert_sql, records, page_size=200)
conn.commit()

print(f"Loaded/Upserted rows: {len(records)}")

Loaded/Upserted rows: 20


In [68]:
# Data check
pd.read_sql("SELECT * FROM news_archive ORDER BY last_seen_at DESC;", conn)

C:\Users\andre\AppData\Local\Temp\ipykernel_88936\1902506266.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM news_archive ORDER BY last_seen_at DESC;", conn)


,story_id,title,url,domain,author,published_at_utc,score,comment_count,first_seen_at,last_seen_at
0,46564696,"Allow me to introduce, the Citroen C15",https://eupolicy.social/@jmaris/11586059523809...,eupolicy.social,colinprince,2026-01-10 11:12:15+00:00,70,35,2026-01-10 18:47:51.061978+00:00,2026-01-10 18:47:51.061978+00:00
1,46565132,"A Eulogy for Dark Sky, a Data Visualization Ma...",https://nightingaledvs.com/dark-sky-weather-da...,nightingaledvs.com,skadamat,2026-01-10 12:23:20+00:00,34,18,2026-01-10 18:47:51.061978+00:00,2026-01-10 18:47:51.061978+00:00
2,46560445,“Erdos problem #728 was solved more or less au...,https://mathstodon.xyz/@tao/115855840223258103,mathstodon.xyz,cod1r,2026-01-09 22:39:15+00:00,521,284,2026-01-10 18:47:51.061978+00:00,2026-01-10 18:47:51.061978+00:00
3,46524379,Changes to Android Open Source Project,https://source.android.com/,source.android.com,TechTechTech,2026-01-07 09:35:46+00:00,190,112,2026-01-10 18:47:51.061978+00:00,2026-01-10 18:47:51.061978+00:00
4,46565281,All my new code will be closed-source from now on,https://twitter.com/MarcJSchmidt/status/200968...,twitter.com,mvelbaum,2026-01-10 12:51:25+00:00,37,33,2026-01-10 18:47:51.061978+00:00,2026-01-10 18:47:51.061978+00:00
5,46511555,Greenland sharks maintain vision for centuries...,https://phys.org/news/2026-01-eye-greenland-sh...,phys.org,pseudolus,2026-01-06 12:44:52+00:00,158,62,2026-01-10 18:47:51.061978+00:00,2026-01-10 18:47:51.061978+00:00
6,46564762,New information extracted from Snowden PDFs th...,https://libroot.org/posts/going-through-snowde...,libroot.org,libroot,2026-01-10 11:23:59+00:00,13,4,2026-01-10 18:47:51.061978+00:00,2026-01-10 18:47:51.061978+00:00
7,46557489,JavaScript Demos in 140 Characters,https://beta.dwitter.net,beta.dwitter.net,themanmaran,2026-01-09 18:48:30+00:00,281,53,2026-01-10 18:47:51.061978+00:00,2026-01-10 18:47:51.061978+00:00
8,46556695,How Markdown took over the world,https://www.anildash.com/2026/01/09/how-markdo...,anildash.com,zdw,2026-01-09 17:52:20+00:00,321,246,2026-01-10 18:47:51.061978+00:00,2026-01-10 18:47:51.061978+00:00
9,46560217,Start your meetings at 5 minutes past,https://philipotoole.com/start-your-meetings-a...,philipotoole.com,otoolep,2026-01-09 22:19:03+00:00,172,157,2026-01-10 18:47:51.061978+00:00,2026-01-10 18:47:51.061978+00:00
